In [ ]:
import music21
import pathlib

from thesession.converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [ ]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=True) # For using fusion to manage >10s clips
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [11]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [12]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [13]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [14]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [15]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [16]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.8502, 0.8558, 0.8473],
        [0.8502, 1.0000, 0.9414, 0.8412],
        [0.8558, 0.9414, 1.0000, 0.8524],
        [0.8473, 0.8412, 0.8524, 1.0000]], device='cuda:0')

# Dataset

In [1]:
import pathlib
import numpy as np
import pandas as pd

from thesession.dataset import TheSessionDataset

path = pathlib.Path("audio_flac")
tunes = [p.stem for p in path.iterdir() if p.is_dir()]
tunes.sort()

tunes_df = pd.DataFrame({"tune": tunes.copy(), "dataset": None})

prng = np.random.default_rng(42)
tunes = prng.permutation(tunes)

tunes_df.loc[tunes_df["tune"].isin(tunes[0 : int(0.15 * len(tunes))]), "dataset"] = "test"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.15 * len(tunes)) : int(0.3 * len(tunes))]), "dataset"] = "validation"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.3 * len(tunes)) :]), "dataset"] = "train"


tunes_df.to_csv("dataset.csv")

In [2]:
test_subset = tunes_df.loc[tunes_df["dataset"] == "test", "tune"]
val_subset = tunes_df.loc[tunes_df["dataset"] == "validation", "tune"]
train_subset = tunes_df.loc[tunes_df["dataset"] == "train", "tune"]

train_dataset = TheSessionDataset("audio_flac", subset=train_subset, sampling_rate=48000, fmt=".flac")
val_dataset = TheSessionDataset("audio_flac", subset=val_subset, sampling_rate=48000, fmt=".flac")
test_dataset = TheSessionDataset("audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac")

print("Training data", len(train_dataset))
print("Validation data", len(val_dataset))
print("Test data", len(test_dataset))

Training data 26344
Validation data 5534
Test data 5728


In [3]:
train_dataset[32]

(tensor([-0.0138, -0.0157, -0.0151,  ..., -0.0132, -0.0090, -0.0040]),
 tensor([ 0.0086,  0.0108,  0.0123,  ..., -0.0052, -0.0035, -0.0016]))

In [4]:
train_dataset[32][0].max()

tensor(0.0609)

In [5]:
train_dataset[32][0].min()

tensor(-0.0386)

In [6]:
%%timeit

train_dataset[0]

47 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

In [8]:
import time

start = time.perf_counter()

for i, (x1, x2) in enumerate(train_loader):
    if i < 20:
        print(i, end="\r")
    else:
        break

end = time.perf_counter()
print("Duration: ", end - start)



Duration:  8.030320544996357


# Model

In [1]:
from thesession.model import TheSessionModel

model = TheSessionModel()
model.load_ckpt()

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [2]:
model.toggle_gradients(False)

Disabling gradient for parameter clap_model.model.logit_scale_a
Disabling gradient for parameter clap_model.model.logit_scale_t
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_real.weight
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_imag.weight
Disabling gradient for parameter clap_model.model.audio_branch.logmel_extractor.melW
Disabling gradient for parameter clap_model.model.audio_branch.bn0.weight
Disabling gradient for parameter clap_model.model.audio_branch.bn0.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.mel_conv

In [3]:
model.toggle_gradients(True, ['clap_model.model.audio_projection'])

Enabling gradient for parameter clap_model.model.audio_projection.0.weight
Enabling gradient for parameter clap_model.model.audio_projection.0.bias
Enabling gradient for parameter clap_model.model.audio_projection.2.weight
Enabling gradient for parameter clap_model.model.audio_projection.2.bias


# Entraînement

In [11]:
import pandas as pd

from thesession.dataset import TheSessionDataset
from thesession.model import TheSessionModel
from thesession.training import eval_model, NTXentLoss

# Dataset
dataset = pd.read_csv("dataset.csv")

test_subset = dataset.loc[dataset["dataset"] == "test", "tune"]

test_dataset = TheSessionDataset(
    "audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac"
)

model = TheSessionModel(device="cuda")
criterion = NTXentLoss(temperature=0.05)
model.eval()

with open("testing.csv", "w") as f:
    f.write("\nstep,test_loss,test_accuracy")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Baseline
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nbaseline,{loss},{acc}")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 79.15 seconds
Test Loss: 4.1314 | Test Accuracy: 0.0402 | 


In [13]:
# Pretrained weights
model.load_ckpt()
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\npretrain,{loss},{acc}")

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 78.15 seconds
Test Loss: 5.5868 | Test Accuracy: 0.0102 | 


In [17]:
model.load("models/step1_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep1,{loss},{acc}")


Duration of evaluation on test data: 76.79 seconds
Test Loss: 1.3755 | Test Accuracy: 0.8455 | 


In [18]:
model.load("models/step2_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep2,{loss},{acc}")

Duration of evaluation on test data: 77.31 seconds
Test Loss: 1.2046 | Test Accuracy: 0.8943 | 


In [19]:
model.load("models/step3_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep3,{loss},{acc}")

Duration of evaluation on test data: 78.79 seconds
Test Loss: 0.3282 | Test Accuracy: 0.9010 | 


In [20]:
model.load("models/step4_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep4,{loss},{acc}")

Duration of evaluation on test data: 78.08 seconds
Test Loss: 0.1179 | Test Accuracy: 0.9696 | 


In [21]:
model.load("models/step5_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep5,{loss},{acc}")

Duration of evaluation on test data: 78.67 seconds
Test Loss: 0.1029 | Test Accuracy: 0.9763 | 


In [22]:
model.load("models/step6_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep6,{loss},{acc}")

Duration of evaluation on test data: 76.33 seconds
Test Loss: 0.0330 | Test Accuracy: 0.9923 | 


# Retriever

In [1]:
import os
import dotenv

import pandas as pd

from thesession.retriever import TheSessionRetriever, get_database_url

dotenv.load_dotenv("thesession-db/.env")

url = get_database_url(
    os.getenv("POSTGRES_USER"),
    os.getenv("POSTGRES_PASSWORD"),
    port=15432,
    database=os.getenv("POSTGRES_DB")
)

retriever = TheSessionRetriever(url, "models/step6_best.pt", backend="soundfile")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
retriever("audio_flac/1_cooleys/36371_0.flac", limit=10)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,1,36384,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 14\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.940252
1,1,36379,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 9\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.938674
2,1,36371,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 1\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.926875
3,1,36383,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 13\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.925647
4,1,36389,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 19\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.914596
5,1,36373,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 3\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.909853
6,1,36395,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 25\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.893879
7,1,36386,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 16\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.892249
8,1,36392,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 22\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.890966
9,17758,36231,Fiddler Leahy’s,None,https://thesession.org/tunes/17758,reel,1,X: 1\nT: Fiddler Leahy's\nR: reel\nM: 4/4\nL: ...,0.882060


In [3]:
retriever("audio_flac/8_the_banshee/32850_1.flac", limit=10)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,8,32850,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 5\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.933319
1,8,32864,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 19\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.893997
2,8,32858,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 13\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.869655
3,8,32849,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 4\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.861172
4,24360,46018,The Rabbit And The Lamb,Crisdean MacDonald,https://thesession.org/tunes/24360,jig,7,X: 2\nT: The Rabbit And The Lamb\nR: jig\nM: 6...,0.840943
5,8,32846,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 1\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.822947
6,17942,55451,John Joe Moroney’s Favourite,Dermot Lernihan,https://thesession.org/tunes/17942,reel,6,X: 1\nT: John Joe Moroney's Favourite\nR: reel...,0.822024
7,8,32847,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 2\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.821859
8,8,32854,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 9\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.817459
9,3484,34596,Adele’s Bath,Jacob Fournel,https://thesession.org/tunes/3484,jig,16,X: 1\nT: Adele's Bath\nR: jig\nM: 6/8\nL: 1/8\...,0.812761


In [5]:
retriever("audio_flac/7346_hommage__edmond_parizeau/3724_4.flac", limit=10)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,7346,3724,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 4\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.918252
1,7346,3725,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 5\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.899184
2,7346,3721,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 1\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.895876
3,7346,3723,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 3\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.856666
4,7346,3722,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 2\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.825461
5,7937,13808,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 2\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.818952
6,7937,13807,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 1\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.818717
7,14494,46477,L’Antre,Fred Guichen,https://thesession.org/tunes/14494,waltz,14,X: 2\nT: L'Antre\nR: waltz\nM: 3/4\nL: 1/8\nK:...,0.799746
8,20083,43476,Bridal Festival Quadrille,None,https://thesession.org/tunes/20083,jig,8,X: 2\nT: Bridal Festival Quadrille\nR: jig\nM:...,0.786909
9,7116,50095,Cuckold Come Out Of The Amrey,None,https://thesession.org/tunes/7116,polka,82,X: 2\nT: Cuckold Come Out Of The Amrey\nR: pol...,0.779676
